In [154]:
import seaborn as sns;
import pandas as pd;
from sklearn.ensemble import RandomForestClassifier as RF
sns.set

# charger le dataset
IBM = pd.read_csv(r"/content/IBM_HR_ATTRITION.csv")
IBM_df = IBM

In [211]:
IBM_df = IBM

In [212]:
IBM_df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,5174,1,Y,Yes,20,4,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,13243,2,Y,No,14,3,4,80,0,17,3,2,9,6,0,8


In [213]:
Y = IBM_df['Attrition'].astype('category').cat.codes
Y

0       1
1       0
2       1
3       0
4       0
       ..
1465    0
1466    0
1467    0
1468    0
1469    0
Length: 1470, dtype: int8

In [214]:
X = IBM_df.drop(['Attrition','EmployeeCount','Education','MonthlyRate','Over18','WorkLifeBalance','DailyRate','PerformanceRating','StockOptionLevel'], axis = 1)
X

,Age,BusinessTravel,Department,DistanceFromHome,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Travel_Rarely,Sales,1,Life Sciences,1,2,Female,94,3,2,Sales Executive,4,Single,5993,8,Yes,11,1,80,8,0,6,4,0,5
1,49,Travel_Frequently,Research & Development,8,Life Sciences,2,3,Male,61,2,2,Research Scientist,2,Married,5130,1,No,23,4,80,10,3,10,7,1,7
2,37,Travel_Rarely,Research & Development,2,Other,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,6,Yes,15,2,80,7,3,0,0,0,0
3,33,Travel_Frequently,Research & Development,3,Life Sciences,5,4,Female,56,3,1,Research Scientist,3,Married,2909,1,Yes,11,3,80,8,3,8,7,3,0
4,27,Travel_Rarely,Research & Development,2,Medical,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,9,No,12,4,80,6,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,Travel_Frequently,Research & Development,23,Medical,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,4,No,17,3,80,17,3,5,2,0,3
1466,39,Travel_Rarely,Research & Development,6,Medical,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,4,No,15,1,80,9,5,7,7,1,7
1467,27,Travel_Rarely,Research & Development,4,Life Sciences,2064,2,Male,87,4,2,Manufacturing Director,2,Married,6142,1,Yes,20,2,80,6,0,6,2,0,3
1468,49,Travel_Frequently,Sales,2,Medical,2065,4,Male,63,2,2,Sales Executive,2,Married,5390,2,No,14,4,80,17,3,9,6,0,8


In [215]:
dummies = pd.get_dummies(X[['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus','EmployeeNumber']], columns=['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus'])
dummies

,EmployeeNumber,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Female,Gender_Male,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single
0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
1,2,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0
2,4,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
3,5,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
4,7,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,2061,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0
1466,2062,0,0,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0
1467,2064,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
1468,2065,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0


In [216]:
X = X.drop(['BusinessTravel','Department','EducationField','Gender','JobRole','MaritalStatus'], axis= 1)

In [217]:
X.OverTime = IBM_df['OverTime'].astype('category').cat.codes

In [218]:
X

,Age,DistanceFromHome,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,NumCompaniesWorked,OverTime,PercentSalaryHike,RelationshipSatisfaction,StandardHours,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,1,2,94,3,2,4,5993,8,1,11,1,80,8,0,6,4,0,5
1,49,8,2,3,61,2,2,2,5130,1,0,23,4,80,10,3,10,7,1,7
2,37,2,4,4,92,2,1,3,2090,6,1,15,2,80,7,3,0,0,0,0
3,33,3,5,4,56,3,1,3,2909,1,1,11,3,80,8,3,8,7,3,0
4,27,2,7,1,40,3,1,2,3468,9,0,12,4,80,6,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,23,2061,3,41,4,2,4,2571,4,0,17,3,80,17,3,5,2,0,3
1466,39,6,2062,4,42,2,3,1,9991,4,0,15,1,80,9,5,7,7,1,7
1467,27,4,2064,2,87,4,2,2,6142,1,1,20,2,80,6,0,6,2,0,3
1468,49,2,2065,4,63,2,2,2,5390,2,0,14,4,80,17,3,9,6,0,8


In [219]:
X = X.merge(dummies, on='EmployeeNumber', how='left')

In [222]:
from sklearn.model_selection import train_test_split

splits = train_test_split(X,Y,
                          train_size = 0.3,
                          stratify = Y,
                          random_state=2)

for each in splits:
  print(each.shape)
X_train, X_test, Y_train, Y_test = splits

(441, 46)
(1029, 46)
(441,)
(1029,)


In [223]:
def accuracy(preds, Y):
  return ((preds == Y).sum() / len(Y)* 100)

# **KNN méthode**

In [262]:
def trainingKNN (n_neighbors):
  knn = KNN(n_neighbors=n_neighbors)
  knn.fit(X_train,Y_train)
  prediction = knn.predict(X_test)
  print(prediction.shape)
  print(prediction)
  print('L\'algorithme KNeighborsClassifier avec n_neighbors =' , n_neighbors , 'est précis à ', accuracy(prediction,Y_test), '%')
  return prediction



In [266]:
prediction = trainingKNN(7)

(1029,)
[0 0 0 ... 0 0 0]
L'algorithme KNeighborsClassifier avec n_neighbors = 7 est précis à  82.6044703595724 %


In [267]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,prediction)

array([[841,  22],
       [157,   9]])

# **SVC méthode**

In [106]:
from sklearn.svm import SVC

In [294]:
svc = SVC(gamma='auto')

def trainingSVC():
  svcTrained = svc.fit(X_train,Y_train)
  prediction = svcTrained.predict(X_test)
  print(prediction)
  print('L\'algorithme Support Vector Classification est précis à ', accuracy(prediction,Y_test), '%')
  return prediction

In [295]:
trainingSVC()

[0 0 0 ... 0 0 0]
L'algorithme Support Vector Classification est précis à  83.86783284742468 %


array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

In [270]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,prediction)

array([[841,  22],
       [157,   9]])

# **Méthode Regression logistique**

In [113]:
from sklearn.linear_model import LogisticRegression as LG

In [322]:
lg = LG(random_state=0, C=1e5, max_iter=10000)

def trainingLG():
  lg = LG(random_state=0, C=1e5, max_iter=10000)
  lgTrained = lg.fit(X_train,Y_train)
  prediction = lgTrained.predict(X_test)
  acc = accuracy(prediction,Y_test)
  print(prediction)
  print('L\'algorithme Support Vector Classification est précis à ', acc, '%')
  return prediction

In [321]:
trainingLG()

[0 0 1 ... 0 0 0]
L'algorithme Support Vector Classification est précis à  84.45092322643343 %


In [323]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,prediction)

array([[798,  65],
       [ 95,  71]])